## SICP 习题 （3.23）解题总结: 双端队列

SICP 习题 3.23 要求我们实现一个双端队列，就是从头部和尾部都可以插入数据和删除数据的队列。

这个习题是习题3.22的延伸，我们只需要修改习题3.22的代码，支持两端的操作就可以了。

支持双端队列的困难在于，每一个新插入的元素都要包含前向指针和后向指针，用简单的序对就不能表示一个元素了。

一个比较工程化的实现是每个元素分配一个(content pointer)，用content保存元素内容，用pointer保存上一个元素的指针。

这种方法相对比较直观，但是其实浪费了很多资源，(content pointer)转化成序对是(content . (pointer . ()),
也就是有两个序对，两个序对有4个指针，只保存了两个数据，另外两个指针一个用来指向poniter, 一个用来指向()，浪费了。

在网上找到一个比较精密的设计，利用单数元素来保存数据，偶数元素来保存反向指针，充分利用了指针资源，缺点就是太精密，不好理解。它的数据结构大概长成这样：

    (() 1 back-ptr 2 back-ptr 3)
    
第一个元素()是反向指针，因为第一个元素的反向没有内容，所以是()
第二个元素1保存的是队列中真实的第一个元素
第三个元素back-ptr是指向第一个元素()的指针
第四个元素2是保存在队列中真实的第二个元素
第五个元素back-ptr是指向第三个元素back-ptr的指针
第六个元素3是保存在队列中真实的第三个元素

具体代码如下，供测试分析：


In [46]:
(define (make-deque)
  
  (let ((front-ptr '())
	(rear-ptr '())
	)

  
    (define (set-front-ptr! item)
      (set! front-ptr item))

    (define (set-rear-ptr! item)
      (set! rear-ptr item))

    (define (empty-deque?)
      (and (null? front-ptr) (null? rear-ptr)))

    (define (front-deque)
      (if (empty-deque?)
	  (error "FRONT called with an empty deque" front-ptr)
	  (cadr front-ptr)))

    (define (rear-deque)
      (if (empty-deque?)
	  (error "REAR called with an empty deque" rear-deque)
	  (cadr rear-ptr)))



    (define (rear-insert-deque! item)
      (let ((new-pair (cons '() (cons item '()))))
	(cond ((empty-deque?)
	       (set-front-ptr! new-pair)
	       (set-rear-ptr! new-pair)
	       'done
	       )
	      (else
	       (set-cdr! (cdr rear-ptr) new-pair)
	       (set-car! new-pair rear-ptr)
	       (set-rear-ptr! new-pair)
	       'done
	       ))))

    (define (front-insert-deque! item)
      (let ((new-pair (cons '() (cons item '()))))
	(cond ((empty-deque?)
	       (set-front-ptr! new-pair)
	       (set-rear-ptr! new-pair)
	       'done)
	      (else
	       (set-cdr! (cdr new-pair) front-ptr)
	       (set-car! front-ptr new-pair)
	       (set-front-ptr! new-pair)
	       'done))))


    (define (front-delete-deque!)
      (cond ((empty-deque?)
	     (error "DELETE! called with an empty deque" front-ptr))
	    (else
	     (if (eq? front-ptr rear-ptr)
		 (begin
		   (set! front-ptr '())
		   (set! rear-ptr '()))
		 (set-front-ptr! (cddr front-ptr)))
	     'done
	     )))


    (define (rear-delete-deque!)
      (cond ((empty-deque?)
	     (error "DELETE! called with an empty deque" front-ptr))
	    (else
	     (if (eq? front-ptr rear-ptr)
		 (begin 
		   (set! front-ptr '())
		   (set! rear-ptr '()))
		 (begin
		   (set-rear-ptr! (car rear-ptr))
		   (set-cdr! (cdr rear-ptr) '())))
	     'done
	     )))

    (define (get-items)
      (define (inter-get queue sequence)
        (if (null? queue)
            '()
            (if (= (remainder sequence 2) 0)
                (if (null? (car queue))
                    (cons '() (inter-get (cdr queue) (+ sequence 1)))
                    (cons 'back-ptr (inter-get (cdr queue) (+ sequence 1))))
                (cons (car queue) (inter-get (cdr queue) (+ sequence 1))))))
      (inter-get front-ptr 0))
    
    

    (define (dispatch m)
      (cond ((eq? m 'front-delete-deque! ) front-delete-deque!)
	    ((eq? m 'front-insert-deque! ) front-insert-deque!)
	    ((eq? m 'rear-delete-deque! ) rear-delete-deque!)
	    ((eq? m 'rear-insert-deque! ) rear-insert-deque!)
	    ((eq? m 'front-deque ) front-deque)
	    ((eq? m 'rear-deque ) rear-deque)
	    ((eq? m 'empty-deque? ) empty-deque?)
	    ((eq? m 'get-items ) get-items)

	    (else (error "Unknown action!" m))))
	    
      

    dispatch))




In [47]:


(define (start-test-3-23)
  (define testing-queue (make-deque))

  (display "it is empty now?") (newline)
  (display ((testing-queue 'empty-deque?)))
  (newline)

  ((testing-queue 'rear-insert-deque!) '1)
  (display ((testing-queue 'get-items)))
 (newline)

  ((testing-queue 'rear-insert-deque!) '2)
  (display ((testing-queue 'get-items)))
 (newline)

  ((testing-queue 'rear-insert-deque!) '3)
  (display ((testing-queue 'get-items)))
 (newline)

  ((testing-queue 'front-insert-deque!) '1a)
  (display ((testing-queue 'get-items)))
 (newline)


  ((testing-queue 'front-insert-deque!) '1a)
  (display ((testing-queue 'get-items)))
 (newline)


  ((testing-queue 'front-insert-deque!) '2a)
  (display ((testing-queue 'get-items)))
 (newline)


  ((testing-queue 'front-insert-deque!) '3a)
  (display ((testing-queue 'get-items)))
 (newline)


  ((testing-queue 'front-delete-deque!) )
  (display ((testing-queue 'get-items)))
 (newline)


  ((testing-queue 'front-delete-deque!) )
  (display ((testing-queue 'get-items)))
 (newline)

  ((testing-queue 'front-delete-deque!) )
  (display ((testing-queue 'get-items)))
 (newline)


  ((testing-queue 'rear-delete-deque!) )
  (display ((testing-queue 'get-items)))
 (newline))


In [48]:
(start-test-3-23)

it is empty now?
True
(() 1)
(() 1 back-ptr 2)
(() 1 back-ptr 2 back-ptr 3)
(() 1a back-ptr 1 back-ptr 2 back-ptr 3)
(() 1a back-ptr 1a back-ptr 1 back-ptr 2 back-ptr 3)
(() 2a back-ptr 1a back-ptr 1a back-ptr 1 back-ptr 2 back-ptr 3)
(() 3a back-ptr 2a back-ptr 1a back-ptr 1a back-ptr 1 back-ptr 2 back-ptr 3)
(back-ptr 2a back-ptr 1a back-ptr 1a back-ptr 1 back-ptr 2 back-ptr 3)
(back-ptr 1a back-ptr 1a back-ptr 1 back-ptr 2 back-ptr 3)
(back-ptr 1a back-ptr 1 back-ptr 2 back-ptr 3)
(back-ptr 1a back-ptr 1 back-ptr 2)
